In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
from PIL import Image
import torchvision.models as models

In [3]:
# Define dataset paths
data_dir = "./data"
metadata_file = os.path.join(data_dir, "HAM10000_metadata.csv")
image_dirs = ["HAM10000_images_part_1", "HAM10000_images_part_2"]
img_size = 224
batch_size = 32
num_epochs = 10



In [4]:
# Data Augmentation & Preprocessing
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])



In [6]:
# Custom Dataset Loader
class SkinDiseaseDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.class_to_idx = {label: idx for idx, label in enumerate(self.data["dx"].unique())}
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_name = self.data.iloc[idx]["image_id"] + ".jpg"
        img_path = None
        for folder in image_dirs:
            potential_path = os.path.join(self.root_dir, folder, img_name)
            if os.path.exists(potential_path):
                img_path = potential_path
                break
        
        if img_path is None:
            raise FileNotFoundError(f"Image {img_name} not found in specified folders.")
        
        image = Image.open(img_path).convert("RGB")
        label = self.class_to_idx[self.data.iloc[idx]["dx"]]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


In [7]:
# Load Dataset
dataset = SkinDiseaseDataset(csv_file=metadata_file, root_dir=data_dir, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [8]:
# Load Pretrained MobileNetV2 as Feature Extractor
model = models.mobilenet_v2(pretrained=True)
num_features = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(128, len(dataset.class_to_idx)),
    nn.Softmax(dim=1)
)


C:\Users\GTEC\AppData\Roaming\Python\Python313\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\GTEC\AppData\Roaming\Python\Python313\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [10]:
# Define Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [11]:
# Training Loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

Epoch 1/10, Loss: 1.4981
Epoch 2/10, Loss: 1.4951
Epoch 3/10, Loss: 1.4949
Epoch 4/10, Loss: 1.4949
Epoch 5/10, Loss: 1.4949
Epoch 6/10, Loss: 1.4945
Epoch 7/10, Loss: 1.4949
Epoch 8/10, Loss: 1.4947
Epoch 9/10, Loss: 1.4956
Epoch 10/10, Loss: 1.4954


In [12]:

# Save Model
torch.save(model.state_dict(), "skin_disease_model.pth")
print("Model saved successfully!")


Model saved successfully!


In [7]:
import torch
from torchvision import models, transforms
from PIL import Image

# 1️⃣ Define the model architecture (ensure it matches the trained model)
class SkinDiseaseModel(torch.nn.Module):
    def __init__(self, num_classes=6):  # Adjust based on trained model
        super(SkinDiseaseModel, self).__init__()
        self.model = models.mobilenet_v2(pretrained=False)  # Ensure same architecture
        self.model.classifier[1] = torch.nn.Linear(self.model.last_channel, num_classes)

    def forward(self, x):
        return self.model(x)

# 2️⃣ Initialize model and load saved weights
model = SkinDiseaseModel(num_classes=6)  # Adjust classes
model.load_state_dict(torch.load("skin_disease_model.pth", map_location=torch.device('cpu')), strict=False)

model.eval()

# 3️⃣ Define Image Preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Use same normalization as training
])

# 4️⃣ Load and preprocess image
image_path = "data/HAM10000_images_part_2/ISIC_0029316.jpg"
image = Image.open(image_path).convert("RGB")
image = transform(image).unsqueeze(0)

# 5️⃣ Run the Model and Predict
with torch.no_grad():
    output = model(image)
    prediction = torch.argmax(output, dim=1).item()

disease_classes = {
    0: "Eczema",
    1: "Psoriasis",
    2: "Melanoma",
    3: "Vitiligo",
    4: "Basal Cell Carcinoma",
    5: "Acne"
}

disease_name = disease_classes.get(prediction, "Unknown Disease")
print(f"Predicted Skin Disease Class: {disease_name}")


Predicted Skin Disease Class: Melanoma
